In [1]:
using Tetris
include("../core/TetrisAICore.jl")
import .TetrisAICore: loadmodel, predict, Model,
 GameState, MoveState, Action, valid_movement, 
 move, put_mino!, generate_minopos, get_node_list,
  Node, draw_game, init_screen, get_key_state, mysleep,
mino_to_array, AILux, create_model, select_node, vector2array
using CUDA
using JLD2
using HTTP
using CodecZstd
using Serialization
CUDA.math_mode!(CUDA.PEDANTIC_MATH)
module TetrisAICore
module AILux
using CUDA
using Lux, LuxCUDA, NNlib, MLUtils, Zygote
import Lux: gpu_device, cpu_device
gpu = gpu_device()
cpu = cpu_device()
export gpu, cpu
using JLD2, Optimisers
using Statistics, Random
using NamedTupleTools
include("play/layers.jl")
include("play/network.jl")
export QNetwork
end
using .AILux
export QNetwork, gpu, cpu
end
using .TetrisAICore

┌ Info: CUDA is on
└ @ Main.TetrisAICore.AILux c:\work_space\programing\julia\TetrisAI\src\core\ai\lux\Lux.jl:13


In [1]:
using Images

using FileIO

function save_3d_array_as_grid_with_padding(arr::Array{T, 3}, filename::String, padding::Int) where T
    h, w, n = size(arr)
    normed = (arr .- minimum(arr)) / (maximum(arr) - minimum(arr))
    
    grid_h = 8
    grid_w = 16
    if n > grid_h*grid_w
        error("The 3rd dimension size should not be greater than 64 for an $(grid_h)x$(grid_w) grid.")
    end
    
    padded_h = h + 2 * padding
    padded_w = w + 2 * padding
    
    composite_img = zeros(Gray{T}, padded_h * grid_h, padded_w * grid_w)
    
    for i in 1:n
        row = div(i-1, grid_w) + 1
        col = rem(i-1, grid_w) + 1
        slice = normed[:, :, i]
        padded_slice = ones(Gray{T}, padded_h, padded_w)
        padded_slice[padding+1:padding+h, padding+1:padding+w] .= Gray.(slice)
        
        composite_img[(row-1)*padded_h+1:row*padded_h, (col-1)*padded_w+1:col*padded_w] .= padded_slice
    end
    
    save(filename, composite_img)
end

ArgumentError: ArgumentError: Package Images not found in current path.
- Run `import Pkg; Pkg.add("Images")` to install the Images package.

In [2]:
model, ps, st = loadmodel("play/mainmodel.jld2")

(_QNetwork(), (board_net = (conv1 = (weight = Float32[0.0018217613 -0.0033527242 -0.0031203853; -0.025152232 -0.00464052 0.0032640556; -0.02294185 -0.01400224 -0.025925517;;; 0.010449293 0.060330622 0.05733042; 0.018899068 -0.046915542 -0.028226; -0.013604938 -0.03970599 -0.05542145;;;; -0.018107533 0.053495944 0.0036241466; 0.015072697 0.025318662 0.00078205287; 0.0075143157 0.010056653 0.049457982;;; 0.019937702 0.03000039 -0.042876992; 0.013525487 -0.009810779 0.001252229; -0.008162948 0.024091426 0.022012621;;;; -0.03748307 0.0012654532 -0.008020445; 0.010418658 -0.019869758 -0.015677404; 0.008536812 0.007809981 0.00047863994;;; -0.030878846 -0.058756657 0.030409522; -0.059250776 -0.038361933 0.038822416; 0.0061465455 0.030691138 0.028219905;;;; … ;;;; 0.035518635 -0.011148178 -0.03817483; 0.009181177 -0.01039618 -0.038135964; -0.010724771 0.0061769714 0.03651171;;; 0.016245063 -0.014320265 -0.015950125; 0.044751342 -0.004680996 -0.043016504; 0.053220265 -0.020674208 0.0065800543;;

In [3]:
# 24x10
board = [
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    1 1 1 1 0 0 0 0 0 0
    1 1 1 1 1 0 0 0 0 0
    1 1 1 1 1 1 0 0 0 0 
    1 1 1 1 1 1 1 0 1 0 
    1 1 1 1 1 1 1 1 1 0 
    1 1 1 1 1 1 1 1 1 0    
]
nothing

In [4]:
# 24x10
minopos = [
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    1 1 0 0 0 0 0 0 0 0
    1 1 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
]
nothing

In [15]:
for i in 1:7
    combo = 0
    btb = 0
    tspin = 0
    # [i_mino, o_mino, s_mino, z_mino, j_mino, l_mino, t_mino]
    holdnext = zeros(7, 6)
    holdnext[1, 1] = 1
    holdnext[6, 2] = 1
    holdnext[6, 3] = 1
    holdnext[6, 4] = 1
    holdnext[6, 5] = 1
    holdnext[i, 6] = 1
    board_array = reshape(board, (24, 10, 1, 1))
    minopos_array = reshape(minopos, (24, 10, 1, 1))
    combo_array = reshape([combo], (1, 1))
    btb_array = reshape([btb], (1, 1))
    tspin_array = reshape([tspin], (1, 1))
    holdnext_array = reshape(holdnext, (7, 6, 1))
    r, _ = model((board_array, minopos_array, combo_array, btb_array, tspin_array, holdnext_array), ps, st)
    println(r[1])
end
nothing

0.8910629166050172


0.8947638158954957
0.8876966116067451


0.8867214969132191
0.89439129045886


0.8974967332106016


0.9100923726458477


In [7]:
model((board_array, minopos_array, combo_array, btb_array, tspin_array, holdnext_array) , ps , st )

([0.6434566091194045;;], (board_net = (conv1 = NamedTuple(), norm1 = (running_mean = Float32[0.14974095, -0.21898468, -0.014769718, -0.1503807, 0.026078673, -0.024106713, -0.24083887, 0.059361987, -0.003845987, 0.41178003  …  0.045489825, 0.030363152, -0.20738648, -0.020222384, 0.00031150825, -0.0016922948, 0.067168884, 0.0981814, 0.21098882, 0.025728004], running_var = Float32[0.0069063716, 0.008884894, 0.0059582987, 0.0027779972, 0.006650655, 0.004324917, 0.0054720887, 0.0037068252, 0.002145184, 0.00937495  …  0.004223671, 0.0010258047, 0.014236818, 0.0013360955, 0.0061511416, 0.004227165, 0.004984162, 0.0025622132, 0.006768717, 0.00062085147], training = Val{false}()), resblocks = (layer_1 = (resblock = (layer_1 = NamedTuple(), layer_2 = (running_mean = Float32[-0.08709227, -0.012833809, 0.27739304, -0.24877937, 0.021279017, -0.12407305, 0.22637543, 0.09612584, 0.11307646, 0.17361048  …  -0.31004447, -0.17861027, -0.035727594, 0.36581355, 0.16892284, -0.100335374, -0.089076325, -0.0

In [11]:
using NNlib

In [12]:
cat3(args...) = cat(args..., dims=3)
neg(x::AbstractArray{T}) where {T} = convert(T, -1.0) * x .+ convert(T, 1.0)

z = cat3(neg(board_array),neg(minopos_array))
z, _ = model.board_net.conv1(z, ps.board_net.conv1, st.board_net.conv1)
z, _ = model.board_net.norm1(z, ps.board_net.norm1, st.board_net.norm1)
z = swish(z)
for i in 1:15
z, _ = model.board_net.resblocks[i](
    z, ps.board_net.resblocks[i], st.board_net.resblocks[i]
)
end

# z, _ = model.board_net.conv2(z, ps.board_net.conv2, st.board_net.conv2)
# z, st_norm2 = model.board_net.norm2(z, ps.board_net.norm2, st.board_net.norm2)
# z = swish(z)
nothing

In [ ]:
tspinz = z
nothing

In [ ]:
d = tspinz - z
save_3d_array_as_grid_with_padding(d[:, :, :, 1], "conv1.png", 3)

In [ ]:
model.board_net.resblocks